In [22]:
%matplotlib inline
import os
import sys
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy
import glob
import datetime
import geopandas as gp

# here are the GEE declarations. 
import ee
ee.Initialize()
# asset to extract. If image collection, convert to single image with stacked bands
# right now it's an ET image where the bands are named "bYYYY-MM"
# this can be adjusted as we find different datasets we want to mess with
to_extract = ee.Image("users/daviddralle/bessv2")
# specify an extracted variable name
# no / used cause will be used in filename
varname = 'ET [mm month-1]'

# here, define a function that takes the bands of the image
# and returns the datetimes corresponding to each band
# this function will need to be edited for different datasets
def get_datetimes(ft):
    bands = [item for item in ft.keys()]
    datelist = [pd.to_datetime(band[1:]) for band in bands]
    return datelist


sites = gp.read_file('../data/sites.shp')
sites = sites.to_crs(epsg=4326)
fts_list = []
for i,row in sites.iterrows():
    pt = ee.Geometry.Point([row.geometry.centroid.x, row.geometry.centroid.y])
    ft = ee.Feature(pt, {'STATION_NM':row['STATION_NM'], 'gauge_id':row['gauge_id']})
    fts_list.append(ft)

# Extract maximum air temperature daily

In [81]:
minyear = 2000
maxyear = 2018
variable = 'tmax'
tmax_dfs = []

for year in range(minyear, maxyear):
    print('Getting %s for year %s'%(variable,str(year)))
    start_date = str(year) + '-05-01'
    end_date = str(year+1) + '-10-01'
    prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').select(variable).filterDate(str(year),str(year+1))
    dfs = []
    for ft in fts_list:
        print('\tGetting site %s'%ft.get('STATION_NM').getInfo())
        def extract(image):
            val = image.reduceRegions(ee.FeatureCollection([ft]), ee.Reducer.first())
            return val
        extracted = prism.map(extract).flatten().getInfo()
        gauge_ids = []
        vals = []
        dates = []
        for feat in extracted['features']:
            gauge_id = feat['properties']['gauge_id']
            val = feat['properties']['first']
            date = pd.to_datetime(feat['id'].split('_')[0])
            gauge_ids.append(gauge_id), vals.append(val), dates.append(date)  

        df = pd.DataFrame(vals, index=dates, columns=[variable])
        df['gauge_id'] = gauge_ids
        df = df.pivot(columns='gauge_id')
        dfs.append(df)
    tmax_dfs.append(pd.concat(dfs, axis=1))
tmax = pd.concat(tmax_dfs)['tmax']
tmax.to_csv('../data/tmax_prism.csv')

Getting tmax for year 2000
	Getting site CLEAR C NR IDRIA CA
	Getting site DEER C NR FOUNTAIN SPRINGS CA
	Getting site BLACK C NR COPPEROPOLIS CA
	Getting site CRISTIANITOS C AB SAN MATEO C NR SAN CLEMENTE CA
	Getting site SAN LORENZO C AB DON CASTRO RES NR CASTRO V CA
	Getting site SAN MATEO C NR SAN CLEMENTE CA
	Getting site CULL C AB CULL C RES NR CASTRO VALLEY CA
	Getting site SAN RAMON C A SAN RAMON CA
	Getting site KELSEY C NR KELSEYVILLE CA
	Getting site ELDER C NR PASKENTA CA
	Getting site BIG C AB WHITES GULCH NR GROVELAND CA
	Getting site LOS GATOS C AB NUNEZ CYN NR COALINGA CA
	Getting site CANTUA C NR CANTUA CREEK CA
	Getting site LOPEZ C NR ARROYO GRANDE CA
	Getting site SAN LORENZO C BL BITTERWATER C NR KING CITY CA
	Getting site MATTOLE R NR PETROLIA CA
	Getting site SESPE CREEK NEAR WHEELER SPRINGS CA
	Getting site ARROYO VALLE BL LANG CN NR LIVERMORE CA
	Getting site ALAMEDA C AB DIV DAM NR SUNOL CA
	Getting site SALSIPUEDES C NR LOMPOC CA
	Getting site SF EEL R A LEGG

# Extract minimum air temperature daily

In [83]:
minyear = 2000
maxyear = 2018
variable = 'tmin'
tmin_dfs = []

for year in range(minyear, maxyear):
    print('Getting %s for year %s'%(variable,str(year)))
    start_date = str(year) + '-05-01'
    end_date = str(year+1) + '-10-01'
    prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').select(variable).filterDate(str(year),str(year+1))
    dfs = []
    for ft in fts_list:
        print('\tGetting site %s'%ft.get('STATION_NM').getInfo())
        def extract(image):
            val = image.reduceRegions(ee.FeatureCollection([ft]), ee.Reducer.first())
            return val
        extracted = prism.map(extract).flatten().getInfo()
        gauge_ids = []
        vals = []
        dates = []
        for feat in extracted['features']:
            gauge_id = feat['properties']['gauge_id']
            val = feat['properties']['first']
            date = pd.to_datetime(feat['id'].split('_')[0])
            gauge_ids.append(gauge_id), vals.append(val), dates.append(date)  

        df = pd.DataFrame(vals, index=dates, columns=[variable])
        df['gauge_id'] = gauge_ids
        df = df.pivot(columns='gauge_id')
        dfs.append(df)
    tmin_dfs.append(pd.concat(dfs, axis=1))
tmin = pd.concat(tmin_dfs)[variable]
tmin.to_csv('../data/tmin_prism.csv')

# Using tmin and tmax data, calculate potential evapotranspiration using the Hargreaves method

In [99]:
import meteolib as meteo
import evaplib as evap

tmax = pd.read_csv('../data/tmax_prism.csv', parse_dates=True, index_col=0)
tmin = pd.read_csv('../data/tmin_prism.csv', parse_dates=True, index_col=0)
rng = tmax.index
# Using Eqn 50 From Allen (1998) to get solar radiation from max/min temp difference
kRs = 0.18
doy = [rng[i].timetuple().tm_yday for i in range(len(rng))]

dfs = []
for col in tmax.columns:
    # get latitude for this site
    site_latitude = sites.loc[sites.gauge_id==col, 'geometry'].values[0].centroid.y
    N, Rext = meteo.sun_NR(doy,39.666)
    Rext_MJ = Rext/(10.0**6)
    tmaxcurr = tmax[col]
    tmincurr = tmin[col]
    tmeancurr = (tmaxcurr+tmincurr)/2.0
    Rs = kRs*np.sqrt(tmaxcurr-tmincurr)*Rext
    pet = evap.hargreaves(tmincurr, tmaxcurr, tmeancurr, Rext_MJ)
    dfs.append(pd.DataFrame(pet.values, index=pet.index, columns=[col]))
pet = pd.concat(dfs, axis=1)
pet.to_csv('../data/pet_prism_hargreaves.csv')

/anaconda3/envs/py3k/lib/python3.5/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in sqrt
